In [ ]:
# !pip install tensorflow
# !pip install numpy
# !pip install pandas
# !pip install matplotlib


In [ ]:
from pathlib import Path
import imghdr
import os

data_dir = "/hair_types"
image_extensions = [".png", ".jpg", ""]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            os.remove(filepath)
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
            os.remove(filepath)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

directory = "hair_types"
file_count = sum(len(files) for _, _, files in os.walk(directory))

print(f"Total number of files in the directory: {file_count}")


image_size = (64, 64)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "hair_types/",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    labels='inferred',
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "hair_types/",
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size, 
    labels='inferred',
    label_mode='categorical'
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(np.argmax(labels[i])))
        plt.axis("off")
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt


model = Sequential()
model.add(keras.Input(shape=image_size + (3,)))
model.add(layers.Rescaling(1.0 / 255))
model.add(layers.RandomFlip("horizontal"))
model.add(layers.RandomRotation(0.1))

model.add(layers.Conv2D(filters=128, kernel_size=7, strides=(1,1), padding='valid', dilation_rate=1, kernel_regularizer=keras.regularizers.l2(1e-05), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.1))
#model.add(Dropout(0.3))

model.add(layers.Conv2D(filters=192, kernel_size=5, strides=(1,1), padding='valid', dilation_rate=1, kernel_regularizer=keras.regularizers.l2(1e-05), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.SpatialDropout2D(0.1))
#model.add(Dropout(0.3))

model.add(layers.Conv2D(filters=256, kernel_size=3, strides=(1,1), padding='valid', dilation_rate=2, kernel_regularizer=keras.regularizers.l2(1e-05), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.SpatialDropout2D(0.1))
#model.add(Dropout(0.3))

model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(100, activation="relu"))
model.add(layers.BatchNormalization())  # Added Batch Normalization
model.add(layers.Dropout(0.3))  # Uncommented and adjusted Dropout

model.add(layers.Dense(50, activation="relu"))
model.add(layers.BatchNormalization())  # Added Batch Normalization before activation
model.add(layers.Activation("relu"))
model.add(layers.Dropout(0.3))  # Uncommented and adjusted Dropout


model.add(layers.Dense(3, activation="softmax"))

#tf.keras.utils.plot_model(model, to_file='model_test.png', show_shapes=True)

epochs = 150

model.compile(
    optimizer=keras.optimizers.Adam(0.1),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)



In [ ]:
# Define the early stopping callback
early_stopping = EarlyStopping(patience=15, restore_best_weights=True)

# Train the model with early stopping
#model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[early_stopping])

# Assuming train_ds and val_ds are defined and the model is compiled
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[early_stopping])

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# Plot the training and validation loss
ax1.plot(history.history['loss'], label='Training Loss')
ax1.plot(history.history['val_loss'], label='Validation Loss')
ax1.set_title('Training and Validation Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss')
ax1.legend()

# Plot the training and validation accuracy
ax2.plot(history.history['accuracy'], label='Training Accuracy')
ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax2.set_title('Training and Validation Accuracy')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy')
ax2.legend()

# Adjust the spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Show the combined graph
plt.show()

In [ ]:
img = keras.preprocessing.image.load_img(
    "hair_types/Curly_Hair/02dac897d1dec9ba8c057a11d041ada8--layered-natural-hair-natural-black-hairstyles.jpg", target_size=image_size
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

predictions = model.predict(img_array)
print(
    "This image is %.2f percent curly hair, %.2f percent straight hair, and %.2f percent wavy hair."
    % tuple(predictions[0])
)
